In [1]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
from train_epoch.train import train_ch6

In [2]:
def conv_block(in_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(in_channels), nn.ReLU(),
        nn.Conv2d(in_channels, num_channels, kernel_size=3, padding=1)
    )

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, in_channels, num_channels):
        super(DenseBlock,self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels * i + in_channels, num_channels))
        self.net = nn.Sequential(*layer)
    
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)
        return X

In [4]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4 ,3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

In [5]:
def transition_block(in_channels, num_channels):
    return nn.Sequential(
        nn.BatchNorm2d(in_channels), nn.ReLU(),
        nn.Conv2d(in_channels, num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )

In [6]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

In [7]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [8]:
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]
blks = []
for i, num_convs in enumerate(num_convs_in_dense_blocks):
    blks.append(DenseBlock(num_convs, num_channels, growth_rate))
    num_channels += num_convs * growth_rate
    if i != len(num_convs_in_dense_blocks) - 1:
        blks.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [9]:
net = nn.Sequential(
    b1, *blks,
    nn.BatchNorm2d(num_channels), nn.ReLU(),
    nn.AdaptiveAvgPool2d((1, 1)),
    nn.Flatten(),
    nn.Linear(num_channels, 10)
)

In [10]:
lr, num_epochs, batch_size = 0.1, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu(),
          save_path="models/DenseNet")

training on cuda:0
new model


Epoch 1/10: 100%|██████████| 60000/60000 [00:09<00:00, 6013.18img/s, loss=0.5243, train_acc=0.8165]


epoch 0, loss 0.524, train acc 0.817, test_acc 0.843


Epoch 2/10: 100%|██████████| 60000/60000 [00:09<00:00, 6169.13img/s, loss=0.3026, train_acc=0.8903]


epoch 1, loss 0.303, train acc 0.890, test_acc 0.843


...

epoch 7, loss 0.162, train acc 0.941, test_acc 0.833


Epoch 9/10: 100%|██████████| 60000/60000 [00:09<00:00, 6209.27img/s, loss=0.1494, train_acc=0.9452]


epoch 8, loss 0.149, train acc 0.945, test_acc 0.896
Saved best model checkpoint to models/DenseNet\05-09-15-23-41\best.ckpt with test accuracy 0.896


Epoch 10/10: 100%|██████████| 60000/60000 [00:09<00:00, 6150.76img/s, loss=0.1394, train_acc=0.9492]


epoch 9, loss 0.139, train acc 0.949, test_acc 0.900
loss 0.139, train acc 0.949, test acc 0.900
7921.4 examples/sec on cuda:0
